In [1]:
#i "nuget:https://www.myget.org/F/eerhardt-testfeed/api/v3/index.json"
#r "nuget:illinkanalyzer,0.0.1-alpha1"

In [1]:
using LinkerAnalyzer;
var graph = new ConsoleDependencyGraph();
graph.Load(@"F:\AppModel\linker\t\experiments\linker-dependencies.xml.gz");

In [1]:
graph.ShowStat(verbose: true)

In [1]:
graph.ShowRoots()

In [1]:
class Data
{
    public Node[] nodes;
    public Link[] links;
}
class Node
{
    public string id;
    public string name;
    public int group;
}
class Link
{
    public string source;
    public string target;
    public int value;
}

In [1]:
using LinkerAnalyzer.Core;
using System.Text.RegularExpressions;

private static Data GetGraphVisual(this DependencyGraph graph, string nodeId)
{
    var nodes = new List<Node>();
    var links = new List<Link>();

    VertexData vertex = graph.Vertex(nodeId);
    if (vertex == null)
    {
        Regex regex = new Regex (nodeId);
        foreach (var v in graph.Types) 
        {
            if (regex.Match (v.value) != Match.Empty) 
            {
                EnsureNode(nodes, v.value, group: 1);
                AddLinks(graph, v, nodes, links);
            }
        }
    }
    else
    {
        EnsureNode(nodes, vertex.value, group: 1);
        AddLinks(graph, vertex, nodes, links);
    }
    
    var graphData = new Data();
    graphData.nodes = nodes.ToArray();
    graphData.links = links.ToArray();
    return graphData;
}

private static void AddLinks(DependencyGraph graph, VertexData vertex, List<Node> nodes, List<Link> links)
{
    foreach (int index in vertex.parentIndexes)
    {
        var childVertex = graph.Vertex (index);
        EnsureNode(nodes, childVertex.value, group: 2);
        links.Add(new Link() {source = vertex.value, target = childVertex.value, value = 1});
        
        var currentParent = childVertex;
        while (childVertex.parentIndexes != null)
        {
            childVertex = graph.Vertex(childVertex.parentIndexes [0]);
            EnsureNode(nodes, childVertex.value, group: 2);
            links.Add(new Link() {source = currentParent.value, target = childVertex.value, value = 1});
            currentParent = childVertex;
        }
    }
}

private static void EnsureNode(List<Node> nodes, string id, int group)
{
    if (!nodes.Exists(n => n.id == id))
    {
        nodes.Add(new Node() {id = id, name = GetName(id), group = group});
    }
}

private static string GetName(string id)
{
    string coreName = GetNameCore(id);
    if (coreName.StartsWith("System."))
        coreName = coreName.Substring("System.".Length);
    if (coreName.Length > 20)
        return coreName.Substring(0, 20) + "...";
    return coreName;
}
private static string GetNameCore(string id)
{
    if (id.StartsWith("TypeDef"))
        return GetTypeDefName(id);
    if (id.StartsWith("Method"))
        return GetMethodName(id);
    
    var index = id.IndexOf(':');
    if (index != -1)
    {
        return id.Substring(index + 1);
    }
    return id;
}

private static string GetTypeDefName(string id)
{
    var index = id.IndexOf(':');
    int index2 = id.Length;
    if (index != -1)
    {
        index2 = id.IndexOf(':', index + 1);
        if (index2 != -1)
        {
            return id.Substring(index + 1, index2 - index - 1);
        }
        return id.Substring(index + 1);
    }
    return id;
}

private static string GetMethodName(string id)
{
    var index = id.IndexOf("::");
    if (index != -1)
    {
        index = id.LastIndexOf('.', index);
        var index2 = id.IndexOf('(', index + 1);
        if (index2 != -1)
        {
            return id.Substring(index + 1, index2 - index - 1);
        }
        return id.Substring(index + 1);
    }
    return id;
}

In [1]:
#!js 

notebookScope.plot2 = (sgvSelector, variableName) => {   
    let dtree_require = require.config({
        paths: {
            d3: "https://d3js.org/d3.v5.min"
        }
    });
    dtree_require(["d3"], function (d3) {
        interactive.csharp.getVariable(variableName)
            .then(data => {
            
                drag = simulation => {

                  function dragstarted(d) {
                    if (!d3.event.active) simulation.alphaTarget(0.3).restart();
                    d.fx = d.x;
                    d.fy = d.y;
                  }

                  function dragged(d) {
                    d.fx = d3.event.x;
                    d.fy = d3.event.y;
                  }

                  function dragended(d) {
                    if (!d3.event.active) simulation.alphaTarget(0);
                    d.fx = null;
                    d.fy = null;
                  }

                  return d3.drag()
                      .on("start", dragstarted)
                      .on("drag", dragged)
                      .on("end", dragended);
                };

                var width = 960,
                    height = 600;

                const links = data.links.map(d => Object.create(d));
                const nodes = data.nodes.map(d => Object.create(d));

                const simulation = d3.forceSimulation(nodes)
                  .force("link", d3.forceLink(links).id(d => d.id))
                  .force("charge", d3.forceManyBody())
                  .force("center", d3.forceCenter(width / 2, height / 2));

                let svg = d3.select(sgvSelector)
                    .attr("viewBox", [0, 0, width, height]);

                const link = svg.append("g")
                    .attr("stroke", "#999")
                    .attr("stroke-opacity", 0.6)
                    .selectAll("line")
                    .data(links)
                    .join("line")
                    .attr("stroke-width", d => Math.sqrt(d.value));

                const scale = d3.scaleOrdinal(d3.schemeCategory10);
                const node = svg.append("g")
                    .selectAll("circle")
                    .data(nodes)
                    .join("circle")
                        .attr("r", 10)
                        .attr("fill", function(d, i){ return scale(d.group); })
                        .call(drag(simulation));

                node.append("title")
                    .text(d => d.id);
                    
                const text = svg.append("g")
                    .selectAll("text")
                    .data(nodes)
                    .join("text")
                    .text(d => d.name);
                 
              simulation.on("tick", () => {
                link
                    .attr("x1", d => d.source.x)
                    .attr("y1", d => d.source.y)
                    .attr("x2", d => d.target.x)
                    .attr("y2", d => d.target.y);
                node
                    .attr("cx", d => d.x)
                    .attr("cy", d => d.y);
                text
                    .attr("x", d => d.x)
                    .attr("y", d => d.y - 12);
              });
        });
    });
};

In [1]:
graph.SpaceAnalyzer = new SpaceAnalyzer(@"F:\AppModel\linker\t\experiments");
graph.SpaceAnalyzer.LoadAssemblies (true);

In [1]:
graph.SpaceAnalyzer.GetLargestNodes(5, "Method")

In [1]:
graph.SpaceAnalyzer.GetLargestNodes(5, "TypeDef")

In [1]:
var graphData = graph.GetGraphVisual(@".*Vector.*");
graphData

In [1]:
#!html
<svg id="dataPlot2" width="100%"></svg>

#!js
notebookScope.plot2("svg#dataPlot2", "graphData");